# TwelveLabs Marengo on Amazon Bedrock Workshop

## Part 0: Setup

### Dependencies

In [ ]:
%pip install -r requirements.txt -Uq

In [ ]:
import boto3, botocore
import json
import re
import pandas as pd
import numpy as np
import uuid
import time
from IPython.display import clear_output, HTML, display, JSON
from sklearn.metrics.pairwise import cosine_similarity
from opensearchpy import AWSV4SignerAuth, NotFoundError, OpenSearch, RequestsHttpConnection

### Configure boto3

In [ ]:
AWS_REGION = "us-east-1" # TODO: Replace with your AWS region

In [ ]:
# Initialize AWS session
session = boto3.Session(profile_name='default') # TODO: Replace with your AWS profile

# Initialize AWS clients
bedrock_client = session.client('bedrock-runtime', region_name=AWS_REGION)
s3_client = session.client('s3')


### Configure S3 bucket

In [ ]:
# S3 Configuration
S3_BUCKET_NAME = "<YOUR_S3_BUCKET>" # TODO: Replace with your S3 bucket name
S3_VIDEOS_PATH = "videos"
S3_IMAGES_PATH = "images"
S3_EMBEDDINGS_PATH = "embeddings"

### Enabling model access on Amazon Bedrock

## Part 1: Multimodal Embeddings with Marengo

### Part 1a: What is an embedding?

In [ ]:
# Sample embeddings
sample_embedding_1 = np.random.rand(1, 1024)
sample_embedding_2 = np.random.rand(1, 1024)

df_embedding_1 = pd.DataFrame(sample_embedding_1)
df_embedding_2 = pd.DataFrame(sample_embedding_2)

df_embedding_1


In [ ]:
# Sample video embedding
sample_video_embedding = np.random.rand(5, 1024)
df_video_embedding = pd.DataFrame(sample_video_embedding)
df_video_embedding

### Part 1b: Calculating cosine similarity

In [ ]:
# Cosine similarity between two single segment embeddings
similarity = cosine_similarity(df_embedding_1, df_embedding_2)
pd.DataFrame(similarity)

In [ ]:
# Cosine similarity with a multi-segment embedding
similarities = cosine_similarity(df_video_embedding, df_embedding_1)
pd.DataFrame(similarities)

In [ ]:
# Getting the max similarity and the index of the max similarity
max_similarity = np.max(similarities)
max_similarity_index = np.argmax(similarities)

print(f"Max similarity: {max_similarity}")
print(f"Index of max similarity: {max_similarity_index}")

## Part 2: Building Multimodal Video Search


### Part 2a: Storing videos in S3

#### Set up sample dataset to S3 bucket

In [ ]:
# AWS Account ID for S3 bucket ownership
aws_account_id = session.client('sts').get_caller_identity()["Account"]

print(f"AWS Account ID: {aws_account_id}")
print(f"S3 Bucket: {S3_BUCKET_NAME}")
print(f"S3 Videos Path: {S3_VIDEOS_PATH}")
print(f"S3 Images Path: {S3_IMAGES_PATH}")
print(f"S3 Embeddings Path: {S3_EMBEDDINGS_PATH}")

# Verify bucket access
try:
    s3_client.head_bucket(Bucket=S3_BUCKET_NAME)
    print(f"✅ Successfully connected to S3 bucket: {S3_BUCKET_NAME}")
except Exception as e:
    print(f"❌ Error accessing S3 bucket: {e}")
    print("Please ensure the bucket exists and you have proper permissions.")


#### Netflix Open Content

The [Netflix Open Content](https://opencontent.netflix.com/) is an open source content available under the [Creative Commons Attribution 4.0 International Public License](https://www.google.com/url?q=https%3A%2F%2Fcreativecommons.org%2Flicenses%2Fby%2F4.0%2Flegalcode&sa=D&sntz=1&usg=AOvVaw3DDX6ldzWtAO5wOs5KkByf).

The assets are available for download at: http://download.opencontent.netflix.com/

We will be utilizing a subset of the videos for demonstrating how to utilize the TwelveLabs models on Amazon Bedrock.

In [ ]:
sample_videos = [
    's3://download.opencontent.netflix.com/TechblogAssets/CosmosLaundromat/encodes/CosmosLaundromat_2048x858_24fps_SDR.mp4',
    's3://download.opencontent.netflix.com/TechblogAssets/Meridian/encodes/Meridian_3840x2160_5994fps_SDR.mp4',
    's3://download.opencontent.netflix.com/TechblogAssets/Sparks/encodes/Sparks_4096x2160_5994fps_SDR.mp4'
]

In [ ]:
public_s3_client = boto3.client('s3', config=botocore.client.Config(signature_version=botocore.UNSIGNED))

In [ ]:
def parse_s3_uri(s3_uri: str) -> tuple[str, str]:
    """
    Parses an S3 URI like s3://bucket-name/path/to/object and returns (bucket, key)

    Args:
        s3_uri (str): The S3 URI to parse
        
    Returns:
        tuple[str, str]: The bucket and key
    """
    pattern = r'^s3://([^/]+)/(.+)$'
    match = re.match(pattern, s3_uri)
    if not match:
        raise ValueError(f"Invalid S3 URI format: {s3_uri}")
    return match.group(1), match.group(2)

def copy_public_s3_object_to_private_bucket(public_s3_uri: str, dest_bucket: str, dest_key: str, aws_profile: str = 'default') -> None:
    """
    Copies a public S3 object to a private bucket

    Args:
        public_s3_uri (str): The S3 URI of the public object to copy
        dest_bucket (str): The name of the private bucket to copy to
        dest_key (str): The key of the object to copy to
        aws_profile (str): The AWS profile to use for the authenticated client
    """

    # Parse source bucket and key
    source_bucket, source_key = parse_s3_uri(public_s3_uri)

    # Anonymous client to read public object
    anon_s3 = boto3.client('s3', config=botocore.client.Config(signature_version=botocore.UNSIGNED))

    print(f"Downloading from {public_s3_uri}...")
    response = anon_s3.get_object(Bucket=source_bucket, Key=source_key)
    data = response['Body'].read()

    print(f"Uploading to s3://{dest_bucket}/{dest_key} ...")
    s3_client.put_object(Bucket=dest_bucket, Key=dest_key, Body=data)

    print("✅ Copy completed successfully!")

In [ ]:
# Copy videos to the S3 bucket
for video_uri in sample_videos:
    # Extract the filename from the S3 key
    _, src_key = parse_s3_uri(video_uri)
    filename = src_key.split("/")[-1]
    dest_key = f"{S3_VIDEOS_PATH}/{filename}"
    copy_public_s3_object_to_private_bucket(
        public_s3_uri=video_uri,
        dest_bucket=S3_BUCKET_NAME,
        dest_key=dest_key
    )

### Part 2b: Creating vector embeddings with Marengo on Bedrock

#### Invoking Marengo on Bedrock

In [ ]:
# Marengo model configuration
MODEL_ID = 'twelvelabs.marengo-embed-2-7-v1:0'

In [ ]:
# Helper function to wait for async embedding results
def wait_for_embedding_output(s3_bucket: str, s3_prefix: str, invocation_arn: str, verbose: bool = False) -> list:
    """
    Wait for Bedrock async embedding task to complete and retrieve results

    Args:
        s3_bucket (str): The S3 bucket name
        s3_prefix (str): The S3 prefix for the embeddings
        invocation_arn (str): The ARN of the Bedrock async embedding task

    Returns:
        list: A list of embedding data
        
    Raises:
        Exception: If the embedding task fails or no output.json is found
    """
    
    # Wait until task completes
    status = None
    while status not in ["Completed", "Failed", "Expired"]:
        response = bedrock_client.get_async_invoke(invocationArn=invocation_arn)
        status = response['status']
        if verbose:
            clear_output(wait=True)
            print(f"Embedding task status: {status}")
        time.sleep(5)
    
    if status != "Completed":
        raise Exception(f"Embedding task failed with status: {status}")
    
    # Retrieve the output from S3
    response = s3_client.list_objects_v2(Bucket=s3_bucket, Prefix=s3_prefix)
    
    for obj in response.get('Contents', []):
        if obj['Key'].endswith('output.json'):
            output_key = obj['Key']
            obj = s3_client.get_object(Bucket=s3_bucket, Key=output_key)
            content = obj['Body'].read().decode('utf-8')
            data = json.loads(content).get("data", [])
            return data
    
    raise Exception("No output.json found in S3 prefix")

In [ ]:
# Create text embedding
def create_text_embedding(text_query: str) -> list:
    """
    Create embeddings for text using Marengo on Bedrock

    Args:
        text_query (str): The text query to create an embedding for
        
    Returns:
        list: A list of embedding data
    """
    
    s3_output_prefix = f'{S3_EMBEDDINGS_PATH}/text/{uuid.uuid4()}'
    
    response = bedrock_client.start_async_invoke(
        modelId=MODEL_ID,
        modelInput={
            "inputType": "text",
            "inputText": text_query
        },
        outputDataConfig={
            "s3OutputDataConfig": {
                "s3Uri": f's3://{S3_BUCKET_NAME}/{s3_output_prefix}'
            }
        }
    )
    
    invocation_arn = response["invocationArn"]
    print(f"Text embedding task started: {invocation_arn}")
    
    # Wait for completion and get results
    try:
        embedding_data = wait_for_embedding_output(S3_BUCKET_NAME, s3_output_prefix, invocation_arn)
    except Exception as e:
        print(f"Error waiting for embedding output: {e}")
        return None
    
    return embedding_data

In [ ]:
# Example: Create text embedding
text_query = "two people having a conversation in a car"

print(f"Creating text embedding for query")
text_embedding_data = create_text_embedding(text_query)

print(f"✅ Text embedding created successfully with {len(text_embedding_data)} segment and {len(text_embedding_data[0]['embedding'])} dimensions.")

In [ ]:
# Create video embedding
def create_video_embedding(video_s3_uri: str) -> list:
    """
    Create embeddings for video using Marengo on Bedrock
    
    Args:
        video_s3_uri (str): The S3 URI of the video to create an embedding for
        
    Returns:
        list: A list of embedding data
    """
    
    s3_output_prefix = f'{S3_EMBEDDINGS_PATH}/{S3_VIDEOS_PATH}/{uuid.uuid4()}'
    
    response = bedrock_client.start_async_invoke(
        modelId=MODEL_ID,
        modelInput={
            "inputType": "video",
            "mediaSource": {
                "s3Location": {
                    "uri": video_s3_uri,
                    "bucketOwner": aws_account_id
                }
            }
        },
        outputDataConfig={
            "s3OutputDataConfig": {
                "s3Uri": f's3://{S3_BUCKET_NAME}/{s3_output_prefix}'
            }
        }
    )
    
    invocation_arn = response["invocationArn"]
    print(f"Video embedding task started: {invocation_arn}")
    
    # Wait for completion and get results
    try:
        embedding_data = wait_for_embedding_output(S3_BUCKET_NAME, s3_output_prefix, invocation_arn)
    except Exception as e:
        print(f"Error waiting for embedding output: {e}")
        return None
    
    return embedding_data


In [ ]:
# Example: Create video embedding
videos = s3_client.list_objects_v2(Bucket=S3_BUCKET_NAME, Prefix=S3_VIDEOS_PATH)["Contents"]
video_uri = f"s3://{S3_BUCKET_NAME}/{videos[0]['Key']}"

print(f"Creating embeddings for video: {video_uri}")
video_embedding_data = create_video_embedding(video_uri)

print(f"✅ Video embedding created successfully with {len(video_embedding_data)} segment(s)")

### Part 2c: Creating a vector index in OpenSearch Serverless

#### Configure Amazon Opensearch Serverless Client

In [ ]:
# OpenSearch Serverless configuration
OPENSEARCH_ENDPOINT = "<YOUR_OPENSEARCH_ENDPOINT>"  # TODO: Replace with your OpenSearch endpoint
INDEX_NAME = "video-embeddings-index"

# Create OpenSearch client for Amazon OpenSearch Serverless
service = "aoss"
credentials = session.get_credentials()
auth = AWSV4SignerAuth(credentials, AWS_REGION, service)

os_client = OpenSearch(
    hosts=[{"host": OPENSEARCH_ENDPOINT, "port": 443}],
    http_auth=auth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection,
    pool_maxsize=20,
)

#### Create a new index

In [ ]:
# Create OpenSearch vector index
def create_opensearch_index(os_client: OpenSearch, index_name: str):
    """
    Create a vector index in OpenSearch for storing video embeddings

    Args:
        os_client (OpenSearch): The OpenSearch client
        index_name (str): The name of the index to create

    Returns:
        None
    """
    
    if os_client.indices.exists(index=index_name):
        print(f"Index '{index_name}' already exists.")
        return
    
    index_body = {
        "settings": {
            "index": {
                "knn": True,
                "number_of_shards": 1,
            }
        },
        "mappings": {
            "properties": {
                "embedding": {
                    "type": "knn_vector",
                    "dimension": 1024,
                    "method": {
                        "engine": "faiss",
                        "name": "hnsw",
                        "space_type": "cosinesimil",
                    },
                },
                "start_time": {"type": "float"},
                "end_time": {"type": "float"},
                "video_id": {"type": "keyword"},
                "segment_text": {"type": "text"},
                "embedding_option": {"type": "keyword"}
            }
        },
    }
    
    os_client.indices.create(index=index_name, body=index_body)
    print(f"✅ Index '{index_name}' created successfully.")

# Create the index
create_opensearch_index(os_client, INDEX_NAME)


#### Bulk process videos in S3 with Marengo

In [ ]:
# Index video embeddings in OpenSearch
def index_video_embeddings(os_client: OpenSearch, index_name: str, video_embeddings: list, video_id: str = "sample_video") -> int:
    """
    Index video embeddings into OpenSearch
    
    Args:
        os_client (OpenSearch): The OpenSearch client
        index_name (str): The name of the index to create
        video_embeddings (list): The list of video embeddings
        video_id (str): The id of the video

    Returns:
        int: The number of documents indexed
    """
    
    documents = []
    
    for i, segment in enumerate(video_embeddings):
        document = {
            "embedding": segment["embedding"],
            "start_time": segment["startSec"],
            "end_time": segment["endSec"],
            "video_id": video_id,
            "segment_id": i,
            "embedding_option": segment.get("embeddingOption", "visual-text")
        }
        documents.append(document)
    
    # Bulk index documents
    bulk_data = []
    for doc in documents:
        bulk_data.append({"index": {"_index": index_name}})
        bulk_data.append(doc)
    
    # Convert to bulk format
    bulk_body = "\n".join(json.dumps(item) for item in bulk_data) + "\n"
    
    response = os_client.bulk(body=bulk_body, index=index_name)
    
    if response["errors"]:
        print("Some documents failed to index:")
        for item in response["items"]:
            if "index" in item and "error" in item["index"]:
                print(f"Error: {item['index']['error']}")
    
    return len(documents)

In [ ]:
# Retrieve the list of videos in the s3 bucket and loop through them to create embeddings
videos = s3_client.list_objects_v2(Bucket=S3_BUCKET_NAME, Prefix=S3_VIDEOS_PATH)["Contents"]

for video in videos:
    video_uri = f"s3://{S3_BUCKET_NAME}/{video['Key']}"
    print(f"Creating embeddings for video: {video_uri}")
    video_embedding_data = create_video_embedding(video_uri)

    print(f"✅ Video embedding created successfully with {len(video_embedding_data)} segment(s) from {video['Key']}")

#### Insert embeddings into OpenSearch index

In [ ]:
# Retrieve the list of embedding files in the S3 bucket
embedding_files = s3_client.list_objects_v2(Bucket=S3_BUCKET_NAME, Prefix=f"{S3_EMBEDDINGS_PATH}/{S3_VIDEOS_PATH}").get("Contents", [])

for embedding_file in embedding_files:
    embedding_key = embedding_file["Key"]
    if not embedding_key.endswith("output.json"):
        continue  # Skip non-JSON files

    embedding_obj = s3_client.get_object(Bucket=S3_BUCKET_NAME, Key=embedding_key)
    content = embedding_obj['Body'].read().decode('utf-8')
    embedding_data = json.loads(content).get("data", [])

    # Use the index_video_embeddings function to index the embedding data into OpenSearch
    num_indexed = index_video_embeddings(os_client, INDEX_NAME, embedding_data, video_id=embedding_key)

    print(f"✅ Indexed {num_indexed} segments from {embedding_key}")

### Part 2d: Querying for multimodal video search

#### Query with text

In [ ]:
# Text Query Search Function
def search_videos_by_text(query_text, top_k=5):
    """Search for video segments using text queries"""
    
    # Generate embedding for the text query
    print(f"Generating embedding for query: '{query_text}'")
    query_embedding_data = create_text_embedding(query_text)
    query_embedding = query_embedding_data[0]["embedding"]
    
    # Search OpenSearch index
    search_body = {
        "query": {
            "knn": {
                "embedding": {
                    "vector": query_embedding,
                    "k": top_k
                }
            }
        },
        "size": top_k,
        "_source": ["start_time", "end_time", "video_id", "segment_id"]
    }
    
    response = os_client.search(index=INDEX_NAME, body=search_body)
    
    print(f"\n✅ Found {len(response['hits']['hits'])} matching segments:")
    results = []
    
    for hit in response['hits']['hits']:
        result = {
            "score": hit["_score"],
            "video_id": hit["_source"]["video_id"],
            "segment_id": hit["_source"]["segment_id"],
            "start_time": hit["_source"]["start_time"],
            "end_time": hit["_source"]["end_time"]
        }
        results.append(result)
        
        print(f"  Score: {result['score']:.4f} | Video: {result['video_id']} | "
              f"Segment: {result['segment_id']} | Time: {result['start_time']:.1f}s - {result['end_time']:.1f}s")
    
    return results


In [ ]:
text_query = "car driving on a road"

In [ ]:
# Example text search
search_results = search_videos_by_text(text_query, top_k=3)

#### Query with image

In [ ]:
# Image Query Search Function


In [ ]:
image_query = ""

In [ ]:
# Example image search

## Part 3: Exercises

### Exercise 1: